In [1]:
#%pip install rasterio

import rasterio

Below, I load an .ovr file from zone 45R and the .tif files for the upper left and bottom right corners of the same zone. I am using one of the lower resolution .ovr pyramid files for efficient processing. 

In [2]:
# Load 45R ovr file
ovr_filename = "s3://w210-poverty-mapper/sentinel2_composite/raw_data/45R/45R_UTM.vrt.ovr.ovr.ovr.ovr.ovr.ovr.ovr"
ovr = rasterio.open(ovr_filename)

In [3]:
# Load top left tif from corresponding ovr file
tl_tif_filename = "s3://w210-poverty-mapper/sentinel2_composite/raw_data/45R/S2_percentile_30_UTM_404-0000069888-0000000000.tif"
tl_tif = rasterio.open(tl_tif_filename)

In [4]:
# Load bottom right tif from corresponding ovr file
br_tif_filename = "s3://w210-poverty-mapper/sentinel2_composite/raw_data/45R/S2_percentile_30_UTM_404-0000000000-0000046592.tif"
br_tif = rasterio.open(br_tif_filename)

The files have the same embedded coordinate reference system (crs).

In [5]:
# Print ovr crs
ovr.crs

CRS.from_epsg(32645)

In [6]:
# Print top left tif crs
tl_tif.crs

CRS.from_epsg(32645)

In [7]:
# Print bottom right tif crs
br_tif.crs

CRS.from_epsg(32645)

You can see below that the top left corner of the .ovr bounding box aligns with the top left corner of the top left .tif file and the bottom right corner of the .ovr file aligns with the bottom right corner of the bottom right .tif file.

In [8]:
# Check ovr bounds
ovr.bounds

BoundingBox(left=194770.0, bottom=2654220.0, right=805230.0, top=3544380.0)

In [9]:
# Check top left tif bounds
tl_tif.bounds

BoundingBox(left=194770.0, bottom=3353100.0, right=427730.0, top=3544380.0)

In [10]:
# Check bottom right tif bounds
br_tif.bounds

BoundingBox(left=660690.0, bottom=2654220.0, right=805230.0, top=2887180.0)

Next we compare pixel values in the .ovr vs. .tif files. We select the top right .tif file for comparison.

In [11]:
# Get first band array for .ovr file
ovr_band1 = ovr.read(1)

In [12]:
# Get first band array for .tif file
tl_tif_band1 = tl_tif.read(1)

The .ovr file has masked RGB values (e.g. zero values) where it extends over the 45R UTM zone boundary. I am starting with row three because the first two rows have masking in the middle of the vector as well due to rough pixel edges.

In [13]:
# Print first row of first RGB band for .ovr file
ovr_band1[3]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0, 1548, 2383, 3521, 1041, 2634, 2393,
       1682, 1787, 1837, 2136, 2168, 2314, 2600, 2502, 2442, 1833, 1868,
       1926, 2291, 1591, 2460, 2432, 2061, 2782, 2655, 2555, 2536, 2392,
       2177, 2081, 2253, 2353, 2276, 2380, 2607, 2670, 2910, 2869, 1841,
       2723, 2667, 2948, 2830, 2669, 2651, 2069, 2297, 2988, 2990, 2659,
       2709, 2454, 2639, 2888, 2196, 1591, 2435, 2027, 2103, 3221, 2890,
       2595, 2469, 2493, 3385, 2689, 3070, 1745, 1934, 1932, 1540, 1323,
       2379, 2180, 2633, 2601, 2504, 2293, 2683, 2395, 2555, 2354, 2140,
       2131, 2431, 2278, 2247, 2454, 2653, 2210, 2573, 2210, 1532, 2121,
       1745, 2397, 2684, 2556, 1144, 2988, 2813,  996, 2681, 1576, 2087,
       2893, 3072, 2644, 2628, 2056, 2854, 2765, 2662, 2988, 2841, 2765,
       2692, 1079, 2972, 2181, 2939, 2282, 2768, 2003, 2407, 2299, 2603,
       2095, 1809, 2118, 1912, 2566, 2156, 1950, 23

The .tif file also has masking on either side of the image where it extends over the UTM zone boundary. I show the beginning, middle, and end of row 300. Since this file has not been compressed, it has a lot more masking at the top of the file, so we index further in to get to rows that aren't all masked. 

In [14]:
# Print first 300 values of first row of RGB band for .tif file
tl_tif_band1[300][0:300]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], d

In [15]:
# Print middle values from first row of RGB band for .tif file
tl_tif_band1[300][11548:11748]

array([2856, 2906, 2903, 2913, 2797, 2693, 2666, 2693, 2467, 2180, 2277,
       2596, 2435, 2396, 2434, 2356, 2456, 2782, 2837, 2742, 2545, 2221,
       2329, 2673, 2706, 2695, 2600, 2587, 2586, 2698, 2714, 2558, 2626,
       2519, 2820, 2830, 2958, 3068, 3059, 3112, 3078, 3077, 3136, 3049,
       2903, 3037, 2932, 2807, 2688, 2646, 2760, 2748, 2773, 2816, 2861,
       2796, 2794, 2792, 2754, 2722, 2708, 2690, 2652, 2684, 2779, 2773,
       2820, 2777, 2600, 2538, 2581, 2533, 2511, 2588, 2625, 2626, 2722,
       2826, 2767, 2702, 2751, 2779, 2767, 2747, 2800, 2752, 2706, 2654,
       2536, 2511, 2517, 2487, 2522, 2482, 2464, 2439, 2436, 2430, 2518,
       2500, 2543, 2833, 2878, 2824, 2945, 2754, 2533, 2626, 2655, 2666,
       2664, 2713, 2759, 2723, 2757, 2713, 2729, 2726, 2697, 2676, 2654,
       2713, 2764, 2797, 2784, 2794, 2828, 2776, 2823, 2837, 2869, 2879,
       2917, 2968, 2931, 2911, 2857, 2800, 2900, 2979, 2972, 2931, 2956,
       2993, 3016, 3011, 3009, 2995, 2977, 2968, 29

In [16]:
# Print last 300 values of first row of RGB band for .tif file
tl_tif_band1[300][-300:]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], d